<a href="https://colab.research.google.com/github/sl20819/Project-Submit-Delete/blob/main/demographics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Demographics").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/dataviz-curriculum/day_1/demographics.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("demographics.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+---+--------------------+---+------------+---------+--------+------------------+---------------+------+-------------+
| id|                name|age|height_meter|weight_kg|children|        occupation|academic_degree|salary|     location|
+---+--------------------+---+------------+---------+--------+------------------+---------------+------+-------------+
|  0|       Darlena Avila| 58|        1.87|       53|       1|     Choreographer|            PhD|    68| South Dakota|
|  1|            Yan Boyd| 65|         1.8|       40|       0|         Cellarman|       Bachelor|    73|     Delaware|
|  2|         Joette Lane| 32|         1.8|       73|       1|Veterinary Surgeon|         Master|    69| South Dakota|
|  3|        Jazmine Hunt| 61|        1.79|       89|       0|            Hawker|            PhD|    88|    Louisiana|
|  4|      Remedios Gomez| 23|        1.64|       51|       2|     Choreographer|       Bachelor|    83|West Virginia|
|  5|        Myung Brewer| 20|        1.68|     

In [4]:
# Print the column names
df.columns

['id',
 'name',
 'age',
 'height_meter',
 'weight_kg',
 'children',
 'occupation',
 'academic_degree',
 'salary',
 'location']

In [15]:
# Print out the first 10 rows
df.limit(10).show()

+---+--------------------+---+------------+---------+--------+------------------+---------------+--------+-------------+
| id|                name|age|height_meter|weight_kg|children|        occupation|academic_degree|Salary1k|     location|
+---+--------------------+---+------------+---------+--------+------------------+---------------+--------+-------------+
|  0|       Darlena Avila| 58|        1.87|       53|       1|     Choreographer|            PhD|      68| South Dakota|
|  1|            Yan Boyd| 65|         1.8|       40|       0|         Cellarman|       Bachelor|      73|     Delaware|
|  2|         Joette Lane| 32|         1.8|       73|       1|Veterinary Surgeon|         Master|      69| South Dakota|
|  3|        Jazmine Hunt| 61|        1.79|       89|       0|            Hawker|            PhD|      88|    Louisiana|
|  4|      Remedios Gomez| 23|        1.64|       51|       2|     Choreographer|       Bachelor|      83|West Virginia|
|  5|        Myung Brewer| 20|  

In [5]:
# Select the age, height_meter, and weight_kg columns and use describe to show the summary statistics
df.describe()

DataFrame[summary: string, id: string, name: string, age: string, height_meter: string, weight_kg: string, children: string, occupation: string, academic_degree: string, salary: string, location: string]

In [6]:
# Print the schema to see the types
df.printSchema

<bound method DataFrame.printSchema of DataFrame[id: string, name: string, age: string, height_meter: string, weight_kg: string, children: string, occupation: string, academic_degree: string, salary: string, location: string]>

In [9]:
# Rename the Salary column to `Salary (1k)` and show only this new column
df = df.withColumnRenamed("Salary","Salary1k")

In [13]:
# Create a new column called `Salary` where the values are the `Salary (1k)` * 1000
df.withColumn("Salary",df["Salary1k"]*1000).show()
# Show the columns `Salary` and `Salary (1k)`

+---+--------------------+---+------------+---------+--------+------------------+---------------+--------+-------------+-------+
| id|                name|age|height_meter|weight_kg|children|        occupation|academic_degree|Salary1k|     location| Salary|
+---+--------------------+---+------------+---------+--------+------------------+---------------+--------+-------------+-------+
|  0|       Darlena Avila| 58|        1.87|       53|       1|     Choreographer|            PhD|      68| South Dakota|68000.0|
|  1|            Yan Boyd| 65|         1.8|       40|       0|         Cellarman|       Bachelor|      73|     Delaware|73000.0|
|  2|         Joette Lane| 32|         1.8|       73|       1|Veterinary Surgeon|         Master|      69| South Dakota|69000.0|
|  3|        Jazmine Hunt| 61|        1.79|       89|       0|            Hawker|            PhD|      88|    Louisiana|88000.0|
|  4|      Remedios Gomez| 23|        1.64|       51|       2|     Choreographer|       Bachelor|